In [19]:
#import relvant packages for conduct topic modeling analysis
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
np.random.seed(2023)

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
from gensim.models import LdaMulticore

In [13]:
from nltk.stem import*
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [14]:
from nltk.corpus import PlaintextCorpusReader
corpus_root = 'MovieReviews' # Folder Name
file_names = PlaintextCorpusReader(corpus_root, '.*',encoding='latin-1')  # wildcard is read all files in the folder
file_names.fileids()  # Get the filenames

['16748.txt',
 '17108.txt',
 '17109.txt',
 '17110.txt',
 '17111.txt',
 '17116.txt',
 '17117.txt',
 '17118.txt',
 '17119.txt',
 '17139.txt',
 '17144.txt',
 '17145.txt',
 '17146.txt',
 '17147.txt',
 '17150.txt',
 '17185.txt',
 '17192.txt',
 '17219.txt',
 '17239.txt',
 '17243.txt',
 '17254.txt',
 '17255.txt',
 '17280.txt',
 '17300.txt',
 '17303.txt',
 '17341.txt',
 '17384.txt',
 '17391.txt',
 '17398.txt',
 '17399.txt',
 '17430.txt',
 '17431.txt',
 '17447.txt',
 '17457.txt',
 '17460.txt',
 '17501.txt',
 '17518.txt',
 '17532.txt',
 '17534.txt',
 '17578.txt',
 '17609.txt',
 '17610.txt',
 '17655.txt',
 '17662.txt',
 '17663.txt',
 '17695.txt',
 '17711.txt',
 '17713.txt',
 '17753.txt',
 '17757.txt',
 '17758.txt',
 '17761.txt',
 '17803.txt',
 '17811.txt',
 '17874.txt',
 '17879.txt',
 '17886.txt',
 '17896.txt',
 '17898.txt',
 '17902.txt',
 '17912.txt',
 '17933.txt',
 '17934.txt',
 '17945.txt',
 '17963.txt',
 '17971.txt',
 '17992.txt',
 '18004.txt',
 '18016.txt',
 '18032.txt',
 '18067.txt',
 '1806

In [27]:
path: str = '/Users/jakebrulato/Documents/GitHub/KDD/Homework_8/MovieReviews'
file_names = os.listdir(path)

doc_contents: list = []
for i, file_name in zip(range(len(file_names)), file_names):
    with open(path + '/' + file_name, encoding="utf8", errors='ignore') as file:
        doc_contents.append((i, file_name, file.read()))

In [33]:
data: DataFrame = pd.DataFrame(doc_contents, columns=['RowNum', 'FileName', 'FileContent'])
data.dropna(subset=['FileContent'], inplace= True)
print(data.head(5))

   RowNum   FileName                                        FileContent
0       0   1912.txt  Date Released: 5/14/93 Running Length: 1:51 Ra...
1       1   1866.txt  Joon (Mary Stuart Masterson) is a young woman ...
2       2   1867.txt  Starring: Lou Diamond Phillips, Jennifer Tilly...
3       3  18087.txt  LOVE IS THE DEVIL (director: John Maybury; Der...
4       4   1911.txt  Starring: Thomas Ian Griffith, Lance Henriksen...


In [34]:
processed_docs = data['FileContent'].map(preprocess)
print(processed_docs[:5])

dictionary: Dictionary = Dictionary(processed_docs)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

0    [date, releas, run, length, rat, violenc, lang...
1    [joon, mari, stuart, masterson, young, woman, ...
2    [star, diamond, phillip, jennif, tilli, toshir...
3    [love, devil, director, john, mayburi, derek, ...
4    [star, thoma, griffith, lanc, henriksen, jam, ...
Name: FileContent, dtype: object


In [35]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print(len(bow_corpus))

180


In [36]:
test_corpus = bow_corpus[0]
for i, word_stat in zip(range(len(test_corpus)), test_corpus):
    # print only the first 5 words
    if i < 5:
        print("Word {} (\"{}\") appears {} times.".format(word_stat[0], dictionary[word_stat[0]], word_stat[1]))

Word 0 ("action") appears 1 times.
Word 1 ("american") appears 3 times.
Word 2 ("arriv") appears 1 times.
Word 3 ("attempt") appears 2 times.
Word 4 ("awar") appears 1 times.
